# ARCTIC automatic reductions
Kolby Weisenburger

March 2016

In [76]:
import astropy.io.fits as pyfits
import numpy as np
import matplotlib.pyplot as plt
import pyfits
import pylab
import os
import re
import pandas as pd
#import pointarray
plt.rcParams['axes.color_cycle'] = ['#2c1e3e','#3a5476','#57839e','#7eb1bc','#b0d9d6']
%matplotlib inline
plt.rcParams["figure.figsize"] = [10,10]

In [90]:
files = [f for f in os.listdir('.') 
         if os.path.isfile(f) if f.endswith(".fits")]

In [91]:
if not os.path.exists('./reduced/cals'):
    os.makedirs('./reduced/cals')
if not os.path.exists('./reduced/data'):
    os.makedirs('./reduced/data')

In [92]:
df = pd.DataFrame(files,columns=['fname'])
df['objtype'] = pd.Series("", index=df.index)
df['filt'] = pd.Series("", index=df.index)
df['exp'] = pd.Series("", index=df.index)
df['objname'] = pd.Series("", index=df.index)

In [93]:
for ff,fname in enumerate(files):
    try:
        df['objtype'][ff] = pyfits.open(fname)[0].header['IMAGETYP']
        df['filt'][ff] = pyfits.open(fname)[0].header['FILTER']
        df['exp'][ff] = pyfits.open(fname)[0].header['EXPTIME']
        df['objname'][ff] = pyfits.open(fname)[0].header['OBJNAME']
    except IOError:    
        print('File corrupt, empty or missing: ' + fname)

In [51]:
def quad_stitch(f):
    datfile = pyfits.getdata(f, header=True)
    dat_raw = datfile[0]
    dat_head = datfile[1]
    
    # ll, ul, lr, ur
    quads = ['DSEC11', 'DSEC21', 'DSEC12', 'DSEC22']

    dat = [[],[],[],[]]
    for i,quad in enumerate(quads):
        head = pyfits.open(f)[0].header[quad]
        idx = re.split('[: ,]',head.rstrip(']').lstrip('['))
        dat[i] = dat_raw[int(idx[2]):int(idx[3]),int(idx[0]):int(idx[1])]
    
    sci_lo = np.concatenate((dat[2], dat[3]), axis = 1)
    sci_up = np.concatenate((dat[0], dat[1]), axis = 1)
    sci = np.concatenate((sci_up, sci_lo), axis = 0)
    
    return [sci,dat_head]

In [52]:
new = quad_stitch('Feige22_r.0094.fits')

In [94]:
df

,fname,objtype,filt,exp,objname
0,bias.0001.fits,Bias,SDSS r,0,
1,bias.0002.fits,Bias,SDSS r,0,
2,bias.0003.fits,Bias,SDSS r,0,
3,bias.0004.fits,Bias,SDSS r,0,
4,bias.0005.fits,Bias,SDSS r,0,
5,bias.0006.fits,Bias,SDSS r,0,
6,bias.0007.fits,Bias,SDSS r,0,
7,bias.0008.fits,Bias,SDSS r,0,
8,bias.0009.fits,Bias,SDSS r,0,
9,Feige22_g.0091.fits,Object,SDSS g,2,Feige 22


In [20]:
### CREATE MASTER BIAS #######################################

bias_idx = df[df['objtype'] == 'Bias'].index.tolist()
biases = np.array([pyfits.getdata(df['fname'][n]) for n in bias_idx])

bias = np.median(biases,axis=0)
pyfits.writeto('./reduced/cals/master_bias.fits',bias,clobber=True)

In [10]:
### CREATE MASTER DARKS ######################################
### these are bias subtracted

dark_flag = 1

times = pd.unique(df.exp.ravel())

for ii in range(0,len(times)):
    dark_idx = df[(df['exp'] == times[ii]) & 
                (df['objtype'] == 'Dark')].index.tolist()
    if len(dark_idx) == 0:
        dark_flag = 0
        continue
      
    #print [(df['fname'][n]) for n in dark_idx]
    darks = np.array([pyfits.getdata(df['fname'][n]) for n in dark_idx]) - bias
    dark_final = np.median(darks,axis=0)
    
    name = './reduced/cals/master_dark_'+str(times[ii])+'.fits'
    pyfits.writeto(name,dark_final,clobber=True)

In [61]:
### CREATE MASTER FLATS ######################################
### these are bias and dark subtracted then normalized

filters = pd.unique(df.filt.ravel())

for ii in range(0,len(filters)):
    flat_idx = df[(df['filt'] == filters[ii]) & 
                   (df['objtype'] == 'Flat')].index.tolist()
    if len(flat_idx) == 0:
        continue
        
    # get the correct master dark (if using darks). if not exact exp time, 
    # scale it from the longest dark frame
    if dark_flag == 1:
        expt = df['exp'][flat_idx[0]]
        try:
            dark = pyfits.getdata('./reduced/cals/master_dark_'+str(expt)+'.fits')
        except IOError:
            scaleto = np.max(df['exp'])
            dark = pyfits.getdata('./reduced/cals/master_dark_'+str(scaleto)+'.fits')
            dark *= (expt/scaleto)
    
        flats = np.array([pyfits.getdata(df['fname'][n]) for n in flat_idx]) - bias - dark
    else:
        #flats = np.array([pyfits.getdata(df['fname'][n]) for n in flat_idx]) - bias #quad#- dark
        flats = np.array([quad_stitch(df['fname'][n])[0] for n in flat_idx]) #- bias #quad#- dark

    
    flat_final = np.median(flats,axis=0)
    flat_final /= np.max(flat_final)

    filts = filters[ii][-1]
    name = './reduced/cals/master_flat_'+filts+'.fits'
    pyfits.writeto(name,flat_final,clobber=True)

/Applications/ureka/variants/common/lib/python2.7/site-packages/pyfits/file.py:372: UserWarning: Overwriting existing file './reduced/cals/master_flat_r.fits'.
  warnings.warn("Overwriting existing file %r." % self.name)
/Applications/ureka/variants/common/lib/python2.7/site-packages/pyfits/file.py:372: UserWarning: Overwriting existing file './reduced/cals/master_flat_g.fits'.
  warnings.warn("Overwriting existing file %r." % self.name)
/Applications/ureka/variants/common/lib/python2.7/site-packages/pyfits/file.py:372: UserWarning: Overwriting existing file './reduced/cals/master_flat_i.fits'.
  warnings.warn("Overwriting existing file %r." % self.name)
/Applications/ureka/variants/common/lib/python2.7/site-packages/pyfits/file.py:372: UserWarning: Overwriting existing file './reduced/cals/master_flat_u.fits'.
  warnings.warn("Overwriting existing file %r." % self.name)
/Applications/ureka/variants/common/lib/python2.7/site-packages/pyfits/file.py:372: UserWarning: Overwriting existin

In [98]:
### REDUCE SCIENCE IMAGES ####################################
# (science_raw - dark) / masterflat

dat_idx = df[df['objtype'] == 'Object'].index.tolist()

for n in dat_idx:
    
    #datfile = pyfits.getdata(df['fname'][n], header=True)
    #dat_raw = datfile[0]
    #dat_head = datfile[1]
    
    
    # turn on quad switch :
    
    datfile = quad_stitch(df['fname'][n])
    dat_raw = datfile[0]
    dat_head = datfile[1]
    
    
    filt = (df['filt'][n])[-1]
    flat = pyfits.getdata('./reduced/cals/master_flat_'+str(filt)+'.fits')
    
    if dark_flag == 1:
        time = df['exp'][n]
        dark = pyfits.getdata('./reduced/cals/master_dark_'+str(time)+'.fits')
        dat = (dat_raw - dark) / flat
    else:
        dat = dat_raw / flat
    
    name = './reduced/data/red_'+df['fname'][n]
    pyfits.writeto(name,dat,clobber=True,header=dat_head)